In [0]:
%pip install pyodbc
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
hostname = "dev-anand-sql-server.database.windows.net"
jdbc_port = "1433"
database = "dev-anand-sql-db"
username = "databricks_user"
password = "<<encrypted>>"
jdbc_url = f"jdbc:sqlserver://{hostname}:{jdbc_port};database={database};encrypt=true;trustServerCertificate=false;loginTimeout=30;"

connection_properties = {
    "user": username,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
mounts = [i.mountPoint for i in dbutils.fs.mounts()]
if '/mnt/adlslanding' in mounts:
    pass
else:
    configs = {"fs.azure.account.key.devanandgen2storage.blob.core.windows.net": "<<encrypted>>"}
    dbutils.fs.mount(
    source="wasbs://landing@devanandgen2storage.blob.core.windows.net/",
    mount_point="/mnt/adlslanding",
    extra_configs=configs)

In [0]:
%sh
#!/bin/bash
# Download the Microsoft signing key
echo "Downloading Microsoft signing key..."
curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
# Add the Microsoft repository for ODBC Driver 18
echo "Adding Microsoft repository..."
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
# Update the package index
echo "Updating package index again..."
sudo apt-get update
# Install the ODBC Driver 17
echo "Installing ODBC Driver 17..."
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
# Install oDBC Driver Manager
# echo "Installing ODBC Driver manager ..."
sudo apt-get -y install unixodbc-dev
# Check if the driver is installed
echo "Checking installed driver..."
dpkg -L msodbcsql17
echo "ODBC Driver installation completed successfully."

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  18316      0 --:--:-- --:--:-- --:--:-- 18547


OK
Adding Microsoft repository...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0   1728      0 --:--:-- --:--:-- --:--:--  1760


deb [arch=amd64,armhf,arm64] https://packages.microsoft.com/ubuntu/22.04/prod jammy mainUpdating package index again...
Hit:1 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...


W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.


Installing ODBC Driver 17...
Reading package lists...
Building dependency tree...
Reading state information...
msodbcsql17 is already the newest version (17.10.6.1-1).
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.
Checking installed driver...
/.
/opt
/opt/microsoft
/opt/microsoft/msodbcsql17
/opt/microsoft/msodbcsql17/include
/opt/microsoft/msodbcsql17/include/msodbcsql.h
/opt/microsoft/msodbcsql17/share
/opt/microsoft/msodbcsql17/share/resources
/opt/microsoft/msodbcsql17/share/resources/en_US
/opt/microsoft/msodbcsql17/share/resources/en_US/msodbcsqlr17.rll
/opt/microsoft/msodbcsql17/etc
/opt/microsoft/msodbcsql17/etc/odbcinst.ini
/opt/microsoft/msodbcsql17/lib64
/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.10.so.6.1
/usr
/usr/lib
/usr/share
/usr/share/doc


In [0]:
import pyodbc

driver = '{ODBC Driver 17 for SQL Server}'

def executeQuery(queryString):
    conn = pyodbc.connect(f"DRIVER={driver};SERVER={hostname};DATABASE={database};UID={username};PWD={password}")
    cursor = conn.cursor()

    cursor.execute(queryString)
    conn.commit()

    cursor.close()
    conn.close()
